In [1]:
import pandas as pd
from glob import glob
import urllib
import json

## Models for all states with various feature combos

In [2]:
better_col_names = {
    'xgbClassifier': 'Simple Classification',
    'xgbOrdinalClassifier': 'Ordinal Classification',
    'xgbCoarseGrainedClassifier': 'Binned(3 Bins) Classification'
}

better_index_name = {
    'babelkw': 'Babelfy KWs',
    'kw': 'YAKE KWs',
    'wikikw': 'Wikifier KWs',
    'st': 'Sentence Transformer',
    'tfidf': 'TF-IDF'
}


In [3]:
results_dir = '../results/results_with_combos/accuracies'
parsed = dict()
with open(results_dir) as f:
    for l in f:
        label, acc = l.split('\t')
        parsed[label] = acc

In [4]:
results = []
for k in parsed:
    model, features, filtered = k.split('.')[0].split('_')
    filtered = filtered == 'True'
    accuracy = float(parsed[k].strip())
    results.append((model, features, filtered, accuracy))
df = pd.DataFrame(results, columns=['model', 'features', 'filtered', 'accuracy'])

In [5]:
df[df.filtered][~df[df.filtered].features.str.contains('+', regex=False)].\
    drop(columns=['filtered']).\
    pivot(index='model', columns='features').\
    transpose().\
    rename(columns=better_col_names, index=better_index_name).\
    style.highlight_max(color='lightgreen').highlight_min(color='red')

In [6]:
df[~df.filtered].\
    drop(columns=['filtered']).\
    pivot(index='model', columns='features').\
    transpose().\
    rename(columns=better_col_names, index=better_index_name).\
    style.highlight_max(color='lightgreen').highlight_min(color='red')

## Models for each state

In [7]:
results_dir = '../results/results_states/accuracies'
parsed = dict()
with open(results_dir) as f:
    for l in f:
        label, acc = l.split('\t')
        parsed[label] = acc

In [8]:
results = []
for k in parsed:
    model, features, filtered, state = k.split('.')[0].split('_')
    filtered = filtered == 'True'
    accuracy = float(parsed[k].strip())
    results.append((model, features, filtered, state, accuracy))
df = pd.DataFrame(results, columns=['model', 'features', 'filtered', 'state', 'accuracy'])

In [9]:
df[~df.filtered].\
    drop(columns=['filtered', 'model']).\
    pivot(index='state', columns='features').\
    transpose().\
    rename(columns=better_col_names, index=better_index_name).\
    style.highlight_max(color='lightgreen').highlight_min(color='red')

In [10]:
df[~df.filtered].\
    drop(columns=['filtered', 'model']).\
    pivot(index='state', columns='features').\
    transpose().\
    rename(columns=better_col_names, index=better_index_name).\
    mean(axis=1)

          features            
accuracy  Babelfy KWs             0.260185
          YAKE KWs                0.232982
          Sentence Transformer    0.292260
          Wikifier KWs            0.245998
dtype: float64

In [11]:
df[~df.filtered].\
    drop(columns=['filtered', 'model']).\
    pivot(index='state', columns='features').\
    transpose().\
    rename(columns=better_col_names, index=better_index_name).\
    transpose().\
    style.highlight_max(color='lightgreen').highlight_min(color='red')

## Top Features for Each State

In [12]:

def go():
    for filename  in glob('../results/results_w_top_features/*top_features*.txt'):
        _, feature_set, _, state = filename.split('/')[-1].replace('[top_features].txt', '').split('_')
        with open(filename) as f:
            for feature_rank, l in enumerate(f):
                feature_name = l.strip()
                yield feature_set, state, feature_name, feature_rank
df = pd.DataFrame(list(go()), columns=['feature_set', 'land', 'feature', 'feature_rank'])
df.loc[df.feature_set == 'kw', 'feature_set'] = 'yake'

In [13]:
def get_simple_feature(babel_feature):
    try:
        f = json.loads(babel_feature)
    except:
        return None
    if f['dbpedia']:
        return 'babel/dbpedia: ' + urllib.parse.unquote(f['dbpedia'][0].split('/')[-1])
    else:
        return 'babel/text: ' + urllib.parse.unquote(f['text'][0])
    
df['simple_feature'] = df.feature_set + ': ' + df.feature    
df.loc[df.feature_set == 'babelkw', 'simple_feature'] = df.feature.apply(get_simple_feature)


In [14]:
df.groupby('simple_feature').feature_rank.median().sort_values(ascending=True)[:50]

simple_feature
babel/dbpedia: Thermodynamic_free_energy     0.0
yake: Grundwissen                            0.5
yake: Blick                                  1.0
babel/dbpedia: Force                         1.5
yake: Energie                                4.0
babel/dbpedia: Fundamental_science           4.0
yake: Körper                                 4.5
wikikw: Elektron                             5.5
babel/text: größerer                         6.0
babel/dbpedia: Animation                     6.5
yake: Licht                                  7.0
babel/dbpedia: Light                         7.5
wikikw: Visuelle Wahrnehmung                 7.5
yake: Wichtigste                             7.5
yake: Abb                                    7.5
babel/dbpedia: Physical_body                 8.0
wikikw: Elektrischer Strom                   8.5
wikikw: Magnetismus                          9.0
yake: Kraft                                  9.0
babel/dbpedia: Motion_(physics)             10.0
yake: